# Ensemble methods

## Questions

### 1
Zespół zawsze ma szansę na osiągnięcie lepszych wyników niż pojedynczy model

### 2
klasyfikacja większościowa polega na zliczeniu klasyfikacji poszczególnych modeli, a następnie wybraniu klasy z największą liczbą głosów

miękkie głosowania może być przeprowadzone jedynie w przypadku modeli które zwracają prawdopodobieństwo przynależności próbki do danej klasy (przeważnie metoda predict_proba w sklearn). Metoda polega na zsumowaniu prawdopodobieństw ze wszystkich modeli oraz wybrania klasy z najwyższym wynikiem.

### 3
Agregacja (bagging) polega na wytrenowaniu zespołu modeli gdzie każdy model jest trenowany na losowym podzbiorze danych treningowych. W tym przypadku przeprowadzamy losowanie ze zwracaniem. Proces trenowania każdego modelu jest niezależny, więc może zostać rozdzielony pomiędzy wiele procesów.

Wklejanie (pasting) - tak samo tylko losowanie bez zwracania. Losowanie musi być przeprowadzone na jednej maszynie.

Wzmacnianie (boosting) - nie, modele ćwiczymy sekwencyjnie. Kolejny model poprawia błędy poprzedniego.

Lasy losowe (random forrest) - tak, poszczególne drzewa mogą być trenowane niezależnie

Kontaminacja (stacking) - niezależnie w obrębie jednej warstwy. Warstwy sekwencyjnie

### 4
Nie musimy tworzyć dodatkowego zbioru walidacyjnego.

### 5
Extra trees są bardziej losowe ponieważ warunek logiczny rozdzielający próbki w węzłach jest wybierany losowo, a nie wyliczany na podstawie wzoru. Jako, że algorytm budujący drzewo decyzyjne jest zachłanny, Extra-Trees mogą uzyskać inny wynik, normalnie nie osiągalny. Extra-Trees szybciej się uczą. Predykcja przebiega w takim samym czasie.

### 6
Można zwiększyć bazowy model lub zwiększyć liczbę estymatorów

### 7
Mniejszy współczynnik uczenia --> więcej modeli --> większe dopasowanie

W przypadku przetrenowania należy zwiększyć współczynnik uczenia

### 8
MNIST

In [1]:
from sklearn.datasets import fetch_openml

mnist = fetch_openml('mnist_784', version=1)
mnist.keys()

dict_keys(['data', 'target', 'frame', 'categories', 'feature_names', 'target_names', 'DESCR', 'details', 'url'])

In [2]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(mnist['data'], mnist['target'], test_size=10_000)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=10_000)
print(f'{len(X_train)}   {len(y_train)}')
print(f'{len(X_test)}   {len(y_test)}')
print(f'{len(X_val)}   {len(y_val)}')

50000   50000
10000   10000
10000   10000


In [3]:
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier
from sklearn.svm import SVC

rf_clf = RandomForestClassifier()
xt_clf = ExtraTreesClassifier()
svm_clf = SVC()
est = [rf_clf, xt_clf, svm_clf]

for clf in est:
    clf.fit(X_train, y_train)

In [7]:
def evaluate_model(model, X_val, y_val):
    y_pred_val = model.predict(X_val)
    print(model.__class__.__name__, accuracy_score(y_val, y_pred_val))

In [9]:
from sklearn.metrics import accuracy_score
for clf in est:
    evaluate_model(clf, X_val, y_val)

RandomForestClassifier 0.9659
ExtraTreesClassifier 0.9695
SVC 0.9773


In [10]:
from sklearn.ensemble import VotingClassifier

bg_clf = VotingClassifier(
    estimators=[
        ('rf', RandomForestClassifier()),
        ('xt', ExtraTreesClassifier()),
        ('svm', SVC())
    ],
    voting = 'hard'
)

bg_clf.fit(X_train, y_train)
evaluate_model(bg_clf, X_val, y_val)

VotingClassifier 0.9715


In [11]:
from sklearn.ensemble import VotingClassifier

bg_soft_clf = VotingClassifier(
    estimators=[
        ('rf', RandomForestClassifier()),
        ('xt', ExtraTreesClassifier()),
        ('svm', SVC(probability=True))
    ],
    voting = 'soft'
)

bg_soft_clf.fit(X_train, y_train)
evaluate_model(bg_soft_clf, X_val, y_val)

VotingClassifier 0.977


In [13]:
y_pred_rf = rf_clf.predict(X_val)
y_pred_xt = xt_clf.predict(X_val)
y_pred_svm = svm_clf.predict(X_val)

In [17]:
import numpy as np
X_mixer = np.vstack((y_pred_rf, y_pred_xt, y_pred_svm)).T
X_mixer.shape

(10000, 3)

In [23]:
from sklearn.ensemble import RandomForestClassifier

rf_mix_clf = RandomForestClassifier()
rf_mix_clf.fit(X_mixer, y_val)

RandomForestClassifier()

In [25]:
y_pred_rf_test = rf_clf.predict(X_test)
y_pred_xt_test = xt_clf.predict(X_test)
y_pred_svm_test = svm_clf.predict(X_test)
X_mixer_test = np.vstack((y_pred_rf_test, y_pred_xt_test, y_pred_svm_test)).T

In [24]:
y_test_pred = rf_mix_clf.predict(X_mixer_test)
accuracy_score(y_test, y_test_pred)

0.9736